# Hashing

### Reading passwords file into list

In [1]:
# Path file passwords2.txt
fname = "C:/Users/sanch/Downloads/passwords2.txt"
# Read and split lines to avoid \n
with open(fname) as f:
    content = f.read().splitlines() 

In [2]:
# Print some results
content[:5]

['OHcv-/U3QI$rdqYTef"D',
 'QtA*.xM$e(+"aO36r&Uo',
 "T;rqw/ou'HN-Pklj6hM*",
 'b%xJ79"A*C5@ehMfS3lu',
 'buI=;LpjBiCm"JS5\'#xy']

In [3]:
# Length of the file passwords2.txt
len(content) 

110000000

## 1. Proccess file hashing the passwords
"AABA" = "AAAB"

We have define our hashing function converting the string into ASCII values, then ordered them and computing the radix-128 form. At last we do the the division method of hashing.

In [4]:
def hash_code(s, radix, mod):
    """
    Takes a string and apply the division method of hashing to the radix-128 form
    h(k) = k % mod
    Args:
        s (str): String to compute
        radix (int): Radix type
        mod (int): Number of buckets
    Returns: 
        result (int): Hash result associated from 0 to mod
    """
    pwr = 1
    result = 0
    list_int_values = [ord(c) for c in s] # Split string and convert each char to ASCII
    list_int_values.sort() # Sort the ASCII numbers in the list
    for char_ascii in list_int_values: # For every list compute string as natural number radix-128
        result += (char_ascii * pwr)
        pwr = (pwr * radix)
    return result % mod

Now, we have to define the hash table size, we have try some values and observed that the number of collisions decreases with the increase of m. That's the reason of choosing 982451653, that is a prime number and not pow of 2.

In [5]:
# Selecting m for the division method
hash_n_buckets = 982451653 
print('m:', hash_n_buckets)

m: 982451653


At this step we compute our hash function with radix 128 and the mod defined before:

In [6]:
# Hash every password
list_hashed = []
for password in content:
    list_hashed.append( hash_code(s=password, radix=128, mod=hash_n_buckets) )

In [8]:
list_hashed[0:5]

[923467939, 199890044, 54920260, 942237609, 129453431]

In [7]:
len(list_hashed)

110000000

In [10]:
len(set(list_hashed))

95073099

As we can see we have **110M** values and **95,073,099 are unique**. So we have only **14,926,901 collisions** that could be false positives or duplicates.

In [13]:
# Write the list into a file
with open('passwords_hashed_prime_high.csv', 'w') as f:
    for item in list_hashed:
        f.write("%s\n" % item)

### Finding duplicates or false positives

In [2]:
# Read the list_hashed obtained before
fname = "passwords_hashed_prime_high.csv"
with open(fname) as f:
    list_hashed = f.read().splitlines()

In [3]:
# Convert string list to int list
list_hashed = list(map(int, list_hashed))

Now we are making a list with the index pointing to the collisions found

In [21]:
index_collision = []
def list_collision(seq):
    seen = set()
    for idx, item in enumerate(seq):
        if item in seen:
            index_collision.append(idx)
        else: 
            seen.add(item)

In [22]:
list_collision(list_hashed)

In [9]:
len(index_collision)

14926901

In [28]:
# Write the list into a file
with open('index_dup_high.csv', 'w') as f:
    for item in index_collision:
        f.write("%s\n" % item)

### Find false positives

In [4]:
# Read the index_collision obtained before
fname = "index_dup_high.csv"
with open(fname) as f:
    index_collision = f.read().splitlines()

In [5]:
# Convert string list to int list
index_collision = list(map(int, index_collision))

In [6]:
len(index_collision)

14926901

We have made a function to count the number of false positives and duplicates as we can see, the thing is that takes to long to compute "list.index" so we aren't able to run all the execution, we have done only 1000 values example and as we can see it takes 9,325 (s). A possible solution could be to use a default dic instead of using list.index to find the index but with our computers we are not able to compute it.

In [12]:
import timeit

def count_d_fd():
    duplicates_count = 0
    false_duplicates_count = 0
    for value_index_coll in index_collision[0:1000]:
# ------------------------------------------------------------------------------------------------------------------------
        value_index_orig = list_hashed.index(list_hashed[value_index_coll]) # Obtain the first index with the same hash value
# ------------------------------------------------------------------------------------------------------------------------
        content_coll = ''.join(sorted(content[value_index_coll]))
        content_set = ''.join(sorted(content[value_index_orig]))
        if content_coll == content_set:
            duplicates_count += 1
        else:
            false_duplicates_count += 1
    print("fp", false_duplicates_count, "d", duplicates_count)
        
timeit.timeit(count_d_fd, number=1)

fp 1000 d 0


9.325398504064879

To conclude, taking into account that we have 10M duplicates as it says the statement in our case we have **14926901 collisions**:
- 10M duplicates 
- 4926901 false positives

## 2. Proccess file hashing the passwords
"AABA" != "AAAB"

In this case we don't have to order the data as we have done before but it is almost the same function and the same proccess:

In [13]:
def hash_code2(s, radix, mod):
    """
    Takes a string and apply the division method of hashing to the radix-128 form
    h(k) = k % mod
    Args:
        s (str): String to compute
        radix (int): Radix type
        mod (int): Number of buckets
    Returns: 
        result (int): Hash result associated from 0 to mod
    """
    pwr = 1
    result = 0
    list_int_values = [ord(c) for c in s] # Split string and convert each char to ASCII
    for char_ascii in list_int_values: # For every list compute string as natural number radix-128
        result += (char_ascii * pwr)
        pwr = (pwr * radix)
    return result % mod

In [14]:
# Selecting m for the division method
hash_n_buckets = 982451653 
print('m:', hash_n_buckets)

m: 982451653


In [ ]:
# Hash every password
list_hashed = []
for password in content:
    list_hashed.append( hash_code2(s=password, radix=128, mod=hash_n_buckets) )